In [81]:
import sys
from functools import reduce
import numpy as np
import cirq
from stabilizer_states import StabilizerStates
from stabilizer_toolkit.decompositions import rank2, validate_decomposition
from stabilizer_toolkit.magic_states import enumerate_ccz, enumerate_t
from stabilizer_toolkit.helpers.unitary import get_tensored_unitary

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
np.set_printoptions(precision=3, linewidth=sys.maxsize, edgeitems=4, threshold=1024, suppress=True) 

In [122]:
S3 = StabilizerStates(3)
print(len(S3))

1080


In [110]:
CCZ = next(enumerate_ccz(3))
decompositions, coeffs = rank2.search_all_stabilizer_states(CCZ, S3)

2023-04-19 20:53:32,077	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|████████████████████████████████████████████████████████████████████████| 582660/582660 [00:36<00:00, 15798.97it/s]


In [111]:
for d, c in zip(decompositions, coeffs):
    validate_decomposition(CCZ, d, c)

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [0.354 0.354 0.354 0.354 0.354 0.354 0.354 0.354]
	+ [-0.707] * [0. 0. 0. 0. 0. 0. 0. 1.]

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [ 0.354  0.354  0.354 -0.354  0.354  0.354  0.354 -0.354]
	+ [0.707] * [0. 0. 0. 1. 0. 0. 0. 0.]

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [ 0.354  0.354  0.354  0.354  0.354 -0.354  0.354 -0.354]
	+ [0.707] * [0. 0. 0. 0. 0. 1. 0. 0.]

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [ 0.354 -0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	+ [0.707] * [0. 1. 0. 0. 0. 0. 0. 0.]

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [ 0.354  0.354  0.354  0.354  0.354  0.354 -0.354 -0.354]
	+ [0.707] * [0. 0. 0. 0. 0. 0. 1. 0.]

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [ 0.354  0.354 -0.354  0.354  0.354  0.354  0.354 -0.354]
	+ 

In [121]:
S3_real = StabilizerStates(3, 'real')
print(len(S3_real))

240


In [119]:
decompositions, coeffs = rank2.search_all_stabilizer_states(CCZ, S3_real)

2023-04-19 20:59:30,126	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████████████████████████████████████████████████████████████████████| 28680/28680 [00:02<00:00, 12898.90it/s]


In [123]:
for d, c in zip(decompositions, coeffs):
    validate_decomposition(CCZ, d, c)

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [0.354 0.354 0.354 0.354 0.354 0.354 0.354 0.354]
	+ [-0.707] * [0. 0. 0. 0. 0. 0. 0. 1.]

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [ 0.354  0.354  0.354 -0.354  0.354  0.354  0.354 -0.354]
	+ [0.707] * [0. 0. 0. 1. 0. 0. 0. 0.]

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [ 0.354  0.354  0.354  0.354  0.354 -0.354  0.354 -0.354]
	+ [0.707] * [0. 0. 0. 0. 0. 1. 0. 0.]

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [ 0.354 -0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	+ [0.707] * [0. 1. 0. 0. 0. 0. 0. 0.]

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [ 0.354  0.354  0.354  0.354  0.354  0.354 -0.354 -0.354]
	+ [0.707] * [0. 0. 0. 0. 0. 0. 1. 0.]

✅ |ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]
	= [1.] * [ 0.354  0.354 -0.354  0.354  0.354  0.354  0.354 -0.354]
	+ 